In [2]:
from pyspark import SparkContext
from pyspark.streaming import StreamingContext
from pyspark.sql import SQLContext
from pyspark.sql.functions import desc
import time
# import subprocess
# subprocess.Popen('python -m TweetRead lit',shell=True)
# seconds = 60*60

In [3]:
sc = SparkContext()
ssc = StreamingContext(sc, 10)
sqlContext = SQLContext(sc)
ssc.checkpoint( "file:///home/majorlazer/spark_logs")

In [4]:
socket_stream = ssc.socketTextStream("127.0.0.1", 5555)

In [5]:
lines = socket_stream.window( 60*60*24 )

In [6]:
# print(type(socket_stream))
# print(type(lines))

In [7]:
from collections import namedtuple
fields = ("tag", "count" )
Tweet = namedtuple('Tweet', fields)

In [8]:
( lines.flatMap( lambda text: text.split( " " ) )
  .filter( lambda word: word.lower().startswith("#") )
  .map( lambda word: ( word.lower(), 1 ) )
  .reduceByKey( lambda a, b: a + b )
  .map( lambda rec: Tweet( rec[0], rec[1] ) )
  .foreachRDD( lambda rdd: rdd.toDF().sort( desc("count") )
              .limit(10).registerTempTable("tweets") ) )

In [9]:
ssc.start() 

In [10]:
sqlContext.sql( 'Select *')

DataFrame[]

In [36]:
top_10_tweets = sqlContext.sql( 'Select tag, count from tweets' )
top_10_df = top_10_tweets.toPandas()
top_10_df

,tag,count
0,#kanyewest,32
1,#yeezy,11
2,#adidas,10
3,#pizzagate,8
4,#shoes,5
5,#boost,5
6,#kanye,5
7,#tshirt,5
8,#cuba,4
9,#cigars,4


In [32]:
# import matplotlib.pyplot as plt
# import seaborn as sn

# from IPython import display


# count = 0
# while count < 10:
# #     time.sleep( 20 )
#     top_10_tweets = sqlContext.sql( 'Select tag, count from tweets' )
#     top_10_df = top_10_tweets.toPandas()
#     display.clear_output(wait=True)
#     sn.plt.figure( figsize = ( 10, 8 ) )
#     sn.barplot( x="count", y="tag", data=top_10_df)
#     sn.plt.show()
#     count = count + 1